In [2]:
!pip install arxiv


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [4]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\9luis\\OneDrive\\Desktop\\Langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
#from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,hf_embeddings)
retriever=vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\9luis\AppData\Local\Temp\ipykernel_28120\3902548930.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\9luis\OneDrive\Desktop\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002731D3C7E00>, search_kwargs={})

In [7]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever, "langsmith_search", "Search for information about LangSmith. For any questions abouot LangSmith, you must choose this tool.")


In [8]:

retriever_tool.name

'langsmith_search'

In [9]:
# Arxiv Tool
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [10]:
tools=[wiki, arxiv, retriever_tool]

In [11]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\9luis\\OneDrive\\Desktop\\Langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions abouot LangSmith, you must choose this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002734F706B60>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vecto

In [12]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()
import os
llm=ChatGroq(model="llama-3.3-70b-versatile", temperature=0, groq_api_key=os.getenv("GROQ_API_KEY"))


In [13]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages
# prompt = hub.pull("rlm/rag-prompt")
# prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [14]:
# Agents
#### will determines the order of use of llms
# from langchain.agents import create_openai_tools_agent
# agent=create_openai_tools_agent(llm, tools, prompt)
# from langchain.agents import create_tool_calling_agent
from langchain.agents import create_react_agent
from langchain_core.prompts import PromptTemplate
#from langchain_community.llms import HuggingFaceHub
react_template = PromptTemplate.from_template("""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
{agent_scratchpad}""")
agent = create_react_agent(llm, tools, react_template)

from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful AI agent."),
#     ("user", "{input}"),
#     ("user", "Use the following tools to help: {agent_scratchpad}")  # ✅ Fixes the error
# ])
# prompt.messages
memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)


C:\Users\9luis\AppData\Local\Temp\ipykernel_28120\1552466613.py:37: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [15]:
## Agent Executor
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_executor=AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True, max_iterations=3,
                             handle_parsing_errors=True, max_execution_time=30,
                             early_stopping_method="generate")
agent_executor

AgentExecutor(memory=ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'), verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever

In [16]:
agent_executor.invoke({"input":"Tell em about LangSmith"})




> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find information about LangSmith, so I should use the LangSmith search tool.

Action: langsmith_search
Action Input: What is LangSmithGet started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your applicati

{'input': 'Tell em about LangSmith',
 'chat_history': [HumanMessage(content='Tell em about LangSmith', additional_kwargs={}, response_metadata={}),
  AIMessage(content='LangSmith is a platform for building production-grade LLM applications. It provides features such as observability, evaluation, and prompt engineering to help developers closely monitor and evaluate their application, ship quickly, and with confidence. The platform allows users to analyze traces, configure metrics and dashboards, and evaluate application performance over production traffic. Additionally, LangSmith provides tools for prompt engineering, enabling developers to find the perfect prompt for their application.', additional_kwargs={}, response_metadata={})],
 'output': 'LangSmith is a platform for building production-grade LLM applications. It provides features such as observability, evaluation, and prompt engineering to help developers closely monitor and evaluate their application, ship quickly, and with con

In [17]:
agent_executor.invoke({"input":"Tell em about Machine Learning"})




> Entering new AgentExecutor chain...
Thought: To provide information about Machine Learning, I should first search for a general overview of the topic. 

Action: wikipedia
Action Input: Machine LearningPage: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genThought: The Wikipedia page provides a good introduction to Machine Learning, but I'd like to explore more about its applications and recent advancements in the field.

Action: arxiv
Action Input: Machine Learning applicationsPublished: 2016-08-28
Title: Proceedings of the 2016 ICML Workshop on #Data4Good: Machine Learning in Social Good Applications
Authors: Kush R. Varshney
Summary: This is the Proceedings of the ICML WoQuestion: Tell me about Machine Learning
Thought: To provide information about Machine Learning, I should first search for a general overview of the topic.

Action: wiki

ValueError: Got unsupported early_stopping_method `generate`